In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import glob, pdb, os, sys, cv2
from ripser import ripser
from persim import plot_diagrams
import matplotlib as mpl
from sklearn import svm
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split,cross_val_score

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf') 

from custom_function import *

In [ ]:
chi_vec = np.round(np.linspace(0,.5,11),2)
hapt_vec = np.round(np.linspace(0,.5,11),2)

## Use 'plane' for sweeping plane filtration and 'flood' for flooding filtration
topo = 'plane'# 'flood'

num_data_sets = 10*len(chi_vec)*len(hapt_vec)

#length of persistence images
feature_len_persim = 2500

#list containing persistence image data
b0_persim_ones = []
b1_persim_ones = []
b0_persim_ramp = []
b1_persim_ramp = []

if topo == "plane":
    num_vecs = 4
elif topo == "flood":
    num_vecs = 1

for i in np.arange(num_vecs):
    b0_persim_ones.append(np.zeros((num_data_sets,feature_len_persim)))
    b1_persim_ones.append(np.zeros((num_data_sets,feature_len_persim)))
    b0_persim_ramp.append(np.zeros((num_data_sets,feature_len_persim)))
    b1_persim_ramp.append(np.zeros((num_data_sets,feature_len_persim)))


#length of Betti curves    
feature_len_Betti = 25

#list containing betti curve data
b0_betti = []
b1_betti = []

for i in np.arange(num_vecs):
    b0_betti.append(np.zeros((num_data_sets,feature_len_Betti)))
    b1_betti.append(np.zeros((num_data_sets,feature_len_Betti)))

#initialize realization, chi, rho, real labels for each simulation    
labels = np.zeros((num_data_sets,))
chi_real_vec = np.zeros((num_data_sets,))
rho_real_vec = np.zeros((num_data_sets,))
real_vec = np.zeros((num_data_sets,))
    
count = 0
for i,chi in enumerate(chi_vec):
    for j,hapt in enumerate(hapt_vec):
        for real in np.arange(10):
            
            
            if "plane" in topo:
                for k,orient in enumerate(['left','right','top','bottom']):

                    #load in persistence images
                    mat = np.load("results/angio_"+topo+"_"+orient+"_persim_ramp_IC_linear_rho_"+str(hapt)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()
                    b0_persim_ramp[k][count,:] = mat['Ip'][0].reshape(-1)
                    b1_persim_ramp[k][count,:] = mat['Ip'][1].reshape(-1)

                    mat = np.load("results/angio_"+topo+"_"+orient+"_persim_ones_IC_linear_rho_"+str(hapt)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()
                    b0_persim_ones[k][count,:] = mat['Ip'][0].reshape(-1)
                    b1_persim_ones[k][count,:] = mat['Ip'][1].reshape(-1)


                    mat = np.load("results/angio_"+topo+"_"+orient+"_Betti_IC_linear_rho_"+str(hapt)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()

                    b0_betti[k][count,:] = mat['b0']
                    b1_betti[k][count,:] = mat['b1']

            elif topo == "flood":
                
                    #load in persistence images
                    mat = np.load("results/angio_"+topo+"_persim_ramp_IC_linear_rho_"+str(hapt)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()
                    b0_persim_ramp[0][count,:] = mat['Ip'][0].reshape(-1)
                    b1_persim_ramp[0][count,:] = mat['Ip'][1].reshape(-1)

                    mat = np.load("results/angio_"+topo+"_persim_ones_IC_linear_rho_"+str(hapt)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()
                    b0_persim_ones[0][count,:] = mat['Ip'][0].reshape(-1)
                    b1_persim_ones[0][count,:] = mat['Ip'][1].reshape(-1)


                    mat = np.load("results/angio_"+topo+"_Betti_IC_linear_rho_"+str(hapt)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()

                    b0_betti[0][count,:] = mat['b0']
                    b1_betti[0][count,:] = mat['b1']


                    
            
            chi_real_vec[count] = chi
            rho_real_vec[count] = hapt
            labels[count] = len(hapt_vec)*i + j
            
            real_vec[count] = real

                
            count+=1

## Run clustering-classification scheme

In [ ]:
'''
###UNCOMMENT THIS PORTION TO PERFORM CLUSTERING FOR ALL INDIVIDUAL DESCRIPTOR VECTORS

X_vec = [b0_persim_ramp,b1_persim_ramp,b0_persim_ones,b1_persim_ones,b0_betti,b1_betti]
title_vec = ["b0_persim_ramp","b1_persim_ramp","b0_persim_ones","b1_persim_ones","b0_betti","b1_betti"]

X_vec = b0_persim_ones + b1_persim_ones  + b0_persim_ramp + b1_persim_ramp + b0_betti + b1_betti
feats = ['persim_ones','persim_ramp','betti']
orients = ['left','right','top','bottom']
title_vec = []
for feat in feats:
    for i in np.arange(2):
        for orient in orients:
            title_vec.append("b"+str(i)+"_"+orient+"_"+feat)

'''

'''
###UNCOMMENT THIS PORTION TO PERFORM CLUSTERING FOR THE 4 BEST INDIVIDUAL FLOODING DESCRIPTOR VECTORS

X_vec = [b1_persim_ramp[0],b1_betti[0],b1_persim_ones[0],b0_betti[0]]
title_vec = ["b1_PIR_flood","b1_BC_flood","b1_PIO_flood","b0_BC_flood"]
title_vec_fig = [r"$PIR_1(K^{flood})$",r"$\beta_1(K^{flood})$",r"$PIO_1(K^{flood})$",r"$\beta_0(K^{flood})$"]
'''

'''
###UNCOMMENT THIS PORTION TO PERFORM CLUSTERING FOR THE 4 BEST DOUBLE FLOODING DESCRIPTOR VECTORS

X_vec = [np.hstack((b0_persim_ones[0],b1_persim_ramp[0])),
        np.hstack((b0_persim_ramp[0],b1_persim_ramp[0])),
        np.hstack((b0_betti[0],b1_betti[0])),
        np.hstack((b0_persim_ones[0],b1_betti[0]))]
title_vec = ["b0_PIO_flood_and_b1_PIR_flood",
             "b0_PIR_flood_and_b1_PIR_flood",
             "b0_BC_flood_and_b1_BC_flood",
             "b0_PIO_flood_and_b1_BC_flood"]

title_vec_fig = [r"$PIO_0(K^{flood})$ & $PIR_1(K^{flood})$",
                 r"$PIR_0(K^{flood})$ & $PIR_1(K^{flood})$",
                 r"$\beta_0(K^{flood})$ & $\beta_1(K^{flood})$",
                 r"$PIO_0(K^{flood})$ & $\beta_1(K^{flood})$"]
'''

'''
###UNCOMMENT THIS PORTION TO PERFORM CLUSTERING FOR THE 4 BEST INDIVIDUAL PLANE SWEEPING DESCRIPTOR VECTORS

X_vec = [b1_persim_ramp[0],b0_persim_ones[0],b1_persim_ones[1],b1_persim_ones[0]]
title_vec = ["b1_PIR_LTR","b0_PIO_LTR","b1_PIO_RTL","b1_PIO_LTR"]
title_vec_fig = [r"$PIR_1(K^{LTR})$",r"$PIO_0(K^{LTR})$",r"$PIO_1(K^{RTL})$",r"$PIO_1(K^{LTR})$"]
'''



###UNCOMMENT THIS PORTION TO PERFORM CLUSTERING FOR THE 4 BEST DOUBLE PLANE SWEEPING DESCRIPTOR VECTORS

X_vec = [np.hstack((b0_persim_ones[0],b0_persim_ones[1])),
np.hstack((b0_persim_ones[1],b1_persim_ramp[0])),
np.hstack((b0_persim_ramp[2],b1_persim_ramp[0])),
np.hstack((b0_persim_ramp[3],b1_persim_ramp[0]))]

title_vec = ["b0_PIO_LTR_and_b0_PIO_RTL",
             "b0_PIO_RTL_and_b1_PIR_LTR",
             "b0_PIR_TTB_and_b1_PIR_LTR",
             "b0_PIR_BTT_and_b1_PIR_LTR"]

title_vec_fig = [r"$PIO_0(K^{LTR})$ & $PIO_0(K^{RTL})$",
                 r"$PIO_0(K^{RTL})$ & $PIR_1(K^{LTR})$",
                 r"$PIR_0(K^{TTB})$ & $PIR_1(K^{LTR})$",
                 r"$PIR_0(K^{BTT})$ & $PIR_1(K^{LTR})$"]


#### perform clustering
for i,X in enumerate(X_vec):
    kmeans_classes,acc,acc_in_sample,centers = clustering_fine_train_test(X,chi_real_vec,rho_real_vec,
                                                real_vec,num_clusters=5,
                                                filename = "param_clustering_"+title_vec[i]+"_"+topo,
                                                title=title_vec_fig[i])

    print(title_vec[i] + ", acc = " + str(acc))



## Creating Accuracy Tables

In [ ]:
'''
### UNCOMMENT THIS PORTION TO PERFORM CLUSTERING FOR ALL INDIVIDUAL DESCRIPTOR VECTORS


X_vec = b0_persim_ones + b1_persim_ones  + b0_persim_ramp + b1_persim_ramp + b0_betti + b1_betti
feats = ['persim\_ones','persim\_ramp','betti']

feats_title = ['PIO','PIR',r'$\beta$']
if "plane" in topo:
    orients = ["LTR","RTL","TTB","BTT"]
elif topo == "flood":
    orients = ["flood"]

title_vec = []
for feat in feats_title:
    for i in np.arange(2):
        for orient in orients:
            title_vec.append(feat + r"$_"+str(i)+"(K^{"+orient+"})$")     

'''


### UNCOMMENT THIS PORTION TO PERFORM CLUSTERING FOR ALL DOUBLE DESCRIPTOR VECTORS

X_vec_ind = b0_persim_ones + b1_persim_ones  + b0_persim_ramp + b1_persim_ramp + b0_betti + b1_betti
feats = ['PIO','PIR','BC']
if "plane" in topo:
    orients = ["LTR","RTL","TTB","BTT"]
elif topo == "flood":
    orients = ["flood"]

title_vec_ind = []
for feat in feats:
    for i in np.arange(2):
        for orient in orients:
            title_vec_ind.append(feat + r"$_"+str(i)+"(K^{"+orient+"})$")         

X_vec = []
title_vec = []
for i in np.arange(len(X_vec_ind)):
    for j in np.arange(i+1,len(X_vec_ind)):
        X_vec.append(np.hstack((X_vec_ind[i],X_vec_ind[j])))
        title_vec.append(title_vec_ind[i] + " \& " + title_vec_ind[j])


create_latex_table_classification_sort(X_vec,title_vec,chi_real_vec,
                                  rho_real_vec,real_vec,num_clusters=5)


## Elbow curve construction

In [ ]:
###
### All doubles of feature vectors
###

X_vec_ind = b0_persim_ones + b1_persim_ones  + b0_persim_ramp + b1_persim_ramp + b0_betti + b1_betti
feats = ['PIO','PIR','BC']
if "plane" in topo:
    orients = ["LTR","RTL","TTB","BTT"]
elif topo == "flood":
    orients = [""]

title_vec_ind = []
for feat in feats:
    for i in np.arange(2):
        for orient in orients:
            title_vec_ind.append(r"$\beta_"+str(i)+"$ "+orient+" "+feat)         

X_vec = []
title_vec = []
for i in np.arange(len(X_vec_ind)):
    for j in np.arange(i+1,len(X_vec_ind)):
        X_vec.append(np.hstack((X_vec_ind[i],X_vec_ind[j])))
        title_vec.append(title_vec_ind[i] + " and " + title_vec_ind[j])

score_vec = np.zeros((9,len(X_vec)))  
for i,X in enumerate(X_vec):


    for num_clusters in np.arange(1,10):
        #### perform clustering
        kmeans_classes,acc,acc_in_sample,_ = clustering_fine_train_test(X,chi_real_vec,rho_real_vec,
                                                real_vec,num_clusters=num_clusters)


        
        score_vec[num_clusters-1,i] = acc

fig = plt.figure(figsize=(8,6))      
ax = fig.add_subplot(111)
ax.plot(np.arange(1,10),score_mean,'b.-',label = title_vec[i])
ax.plot(np.arange(1,10),score_mean + 2*score_std,'b--',label = title_vec[i])
ax.plot(np.arange(1,10),score_mean - 2*score_std,'b--',label = title_vec[i])
ax.set_xlabel("Number of clusters ($k$)")
ax.set_ylabel("OOS Accuracy %")
ax.set_title("Elbow Curve")
ax.set_xticks(np.arange(1,10))
#plt.savefig('figures/elbow_all.pdf', format='pdf')

## Find and depict representative networks

In [ ]:
X = np.hstack((b0_persim_ones[0],b0_persim_ones[1]))

#### perform clustering
kmeans_classes,acc,acc_in_sample,centers = clustering_fine_train_test(X,chi_real_vec,rho_real_vec,
                                                real_vec,num_clusters=5)

#find simulation closest to mean value
chi_means = []
rho_means = []
real_means = []

for i in np.arange(5):
    center_loc = np.argmin([np.linalg.norm(centers[i,:] - row) for row in X])
    
    #underlying params
    chi_means.append(chi_real_vec[center_loc])
    rho_means.append(rho_real_vec[center_loc])
    real_means.append(np.int(real_vec[center_loc]))

print(rho_means)    
print(chi_means)



In [ ]:
x = np.linspace(0,1,201)
y = x.copy()
x2 = np.linspace(0,1,50)
y2 = x2.copy()

font = {'size'   : 8}
plt.rc('font', **font)

cmaplist = [(1.0,1.0,1.0),(1,1,1),(1,0,0)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, N = 3)

count = 0

fig = plt.figure(figsize=(10*1.5,6*1.5))

#this order was manually picked to arrange groups in order
for i in [4,1,3,2,0]:
    
    #find simulation closest to mean value
    center_loc = np.argmin([np.linalg.norm(centers[i,:] - row) for row in X])
    
    chi = chi_means[i]
    rho = rho_means[i]
    real = np.int(real_means[i])
    mat = np.load("results/angio_bio_data_IC_linear_rho_"+str(rho)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()

    print(rho,chi)
    
    #plt.matshow(mat['N'][:,:].T)
    N = mat['N']
    
    ax = fig.add_subplot(2,3,count + 1,adjustable="box")
    cm = ax.contourf(x,y,N.T,vmin=0,vmax=1,cmap=cmap)
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax.set_title("Group " + str(count+1),fontsize=20)
    #if count == 0:
    #    ax.set_title("Representative \n Network",fontsize=20)
    
    mat = np.load("results/angio_"+topo+"_left_persim_ramp_IC_linear_rho_"+str(rho)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()
    mat_RTL = np.load("results/angio_"+topo+"_right_persim_ramp_IC_linear_rho_"+str(rho)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()
    
    
    count += 1
plt.subplots_adjust(left=0.1,bottom=0.04,top=0.9,right = 0.97)
fig.suptitle(r"Mean Cluster Realizations from PIO$_0(K^{LTR})$ & PIO$_0(K^{RTL})$",fontsize=23)
plt.savefig("grouping_b0_PIO_LTR_b0_PIO_RTL.pdf",format="pdf")